<a href="https://colab.research.google.com/github/hishamdewan/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

### Recreate vine_table DataFrames

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3VR960AHLFKDV|          4|            0|          0|   N|                Y|
|R16LGVMFKIUT0G|          5|            0|          0|   N|                Y|
|R1AIMEEPYHMOE4|          5|            1|          1|   N|                Y|
|R1892CCSZWZ9SR|          3|            0|          0|   N|                Y|
|R285P679YWVKD1|          3|            0|          0|   N|                N|
| RLB33HJBXHZHU|          5|            0|          0|   N|                Y|
|R1VGTZ94DBAD6A|          5|            2|          2|   N|                Y|
|R168KF82ICSOHD|          5|            0|          0|   N|                Y|
|R20DIYIJ0OCMOG|          5|            0|          0|   N|                Y|
| RD46RNVOHNZSC|          5|            0|          0|   N|     

In [ ]:
vine_df.count()

792113

In [ ]:
# vine_df is filtered to create a DataFrame or table where there are 20 or more total votes
vine_df_20 = vine_df.filter("total_votes>=20")
vine_df_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
|R3GNSIFV1J2Y2B|          1|           15|         60|   N|                N|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|     

In [ ]:
vine_df_20.count()

18739

In [ ]:
# The data is filtered to create a DataFrame where the percentage of helpful_votes is equal to or greater than 50%
# vine_df_helpful_50 = vine_df_20.filter("helpful_votes>50")
vine_df_helpful_50 = vine_df_20.withColumn('percent_helpful_votes', vine_df_20['helpful_votes']/vine_df_20['total_votes']).filter("percent_helpful_votes>0.50")
vine_df_helpful_50.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|   0.9212121212121213|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|   0.9130434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|                  1.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N|   0.8669354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|   0.9772727272727273|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|   0.9615384615384616|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|   0.9629629629629629|


In [ ]:
vine_df_helpful_50.count()

18135

In [ ]:
# The data is filtered to create a DataFrame or table where there is a Vine review
from pyspark.sql.functions import col
vine_yes = vine_df_helpful_50.filter(col("vine")== "Y")
vine_yes.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|R2BQOD1R0228FN|          3|           17|         26|   Y|                N|   0.6538461538461539|
| RC31RUPFOHBHQ|          4|          102|        117|   Y|                N|   0.8717948717948718|
| REN3N1WITLF1Y|          5|           33|         37|   Y|                N|   0.8918918918918919|
| R71RZQ9UZVG47|          4|           38|         47|   Y|                N|   0.8085106382978723|
|R38NMQBH88HLM6|          4|           18|         24|   Y|                N|                 0.75|
|R33FGX9EE3QVR6|          4|           26|         26|   Y|                N|                  1.0|
|R1KIOIK6WEYE59|          3|           19|         20|   Y|                N|                 0.95|


In [ ]:
vine_yes.count()

136

In [ ]:
# The data is filtered to create a DataFrame or table where there IS NO Vine review
vine_no = vine_df_helpful_50.filter(col("vine")== "N")
vine_no.show()

+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+---------------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|   0.9212121212121213|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|   0.9130434782608695|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|                  1.0|
| RR99CPG695T0I|          5|          215|        248|   N|                N|   0.8669354838709677|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|   0.9772727272727273|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|   0.9615384615384616|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|   0.9629629629629629|


In [ ]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
total_number_of_reviews = vine_df_helpful_50.count()
number_of_5_star_reviews = vine_df_helpful_50.filter("star_rating==5").count()


In [ ]:
unpaid_total_number_of_reviews = vine_no.count()
unpaid_5_star_reviews = vine_no.filter("star_rating==5").count()
percentage_unpaid_5_star_reviews = unpaid_5_star_reviews/unpaid_total_number_of_reviews
percentage_unpaid_5_star_reviews

0.4711372854047447

In [ ]:
paid_total_number_of_reviews = vine_yes.count()
paid_5_star_reviews = vine_yes.filter("star_rating==5").count()
percentage_paid_5_star_reviews = paid_5_star_reviews/paid_total_number_of_reviews
percentage_paid_5_star_reviews

0.5441176470588235

In [37]:
print("Total number of reviews %f" % total_number_of_reviews)
print("Number of 5-star reviews %f" % number_of_5_star_reviews)
print("Total number of unpaid reviews %f" % unpaid_total_number_of_reviews)
print("Total number of Amazon Vine reviews %f" % paid_total_number_of_reviews)
print("Number of 5-star unpaid reviews %f" % unpaid_5_star_reviews)
print("Number of 5-star Amazon Vine reviews %f" % paid_5_star_reviews)
print("Percentage of unpaid 5 star reviews %f" % (percentage_unpaid_5_star_reviews*100), "%")
print("Percentage of Amazon Vine 5 star reviews %f" % (percentage_paid_5_star_reviews*100), "%")

Total number of reviews 18135.000000
Number of 5-star reviews 8554.000000
Total number of unpaid reviews 17999.000000
Total number of Amazon Vine reviews 136.000000
Number of 5-star unpaid reviews 8480.000000
Number of 5-star Amazon Vine reviews 74.000000
Percentage of unpaid 5 star reviews 47.113729 %
Percentage of Amazon Vine 5 star reviews 54.411765 %
